In [1]:
import cobra
import pandas as pd
import json
from custom_functions_scRBA import *

#### Add reactions

In [2]:
mu_s = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
for mu in mu_s:
    # Create a blank model
    model = cobra.Model('test')
    
    # Add metabolites and species
    with open('./species.txt') as f:
        species = f.read().split('\n')
    for m in species:
        met = cobra.Metabolite(m)
        model.add_metabolites(met)
    
    # Add reactions
    df_rxns = pd.read_excel('./scRAM_network_frompy.xlsx')
    df_rxns.index = df_rxns.id.to_list()

    if mu > 0.211:
        with open('./symbolic_stoich/sij_higher_0_211.json') as f:
            coeffs = json.load(f)
    else:
        with open('./symbolic_stoich/sij_lower_0_211.json') as f:
            coeffs = json.load(f)

    coeffs = {k:eval(str(v)) for k,v in coeffs.items()}

    for r in df_rxns.index:
        x = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[r])
        met_dict = dict()
        for k,v in x.items():
            if is_number(v):
                met_dict[k] = v
            else:
                met_dict[k] = coeffs["'" + k + "'.'" + r + "'"]

        rxn = cobra.Reaction(r)
        model.add_reaction(rxn)
        rxn.reaction = build_reaction_equation_from_metabolites_dict_RBA(met_dict, arrow='-->')
    
    # Save model
    cobra.io.save_json_model(model, './models/cobraModel_' + str(mu) + '.json')

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-26


#### Test run time

In [2]:
import time

In [4]:
start_time = time.time()

mu = 0.1

# Create a blank model
model = cobra.Model('test')

# Add metabolites and species
with open('./species.txt') as f:
    species = f.read().split('\n')
for m in species:
    met = cobra.Metabolite(m)
    model.add_metabolites(met)

# Add reactions
df_rxns = pd.read_excel('./scRAM_network_frompy.xlsx')
df_rxns.index = df_rxns.id.to_list()

if mu > 0.211:
    with open('./sij_higher_0_211.json') as f:
        coeffs = json.load(f)
else:
    with open('./sij_lower_0_211.json') as f:
        coeffs = json.load(f)

coeffs = {k:eval(str(v)) for k,v in coeffs.items()}

for r in df_rxns.index:
    x = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[r])
    met_dict = dict()
    for k,v in x.items():
        if is_number(v):
            met_dict[k] = v
        else:
            met_dict[k] = coeffs["'" + k + "'.'" + r + "'"]

    rxn = cobra.Reaction(r)
    model.add_reaction(rxn)
    rxn.reaction = build_reaction_equation_from_metabolites_dict_RBA(met_dict, arrow='-->')

# Save model
cobra.io.save_json_model(model, './models/test_run_time_' + str(mu) + '.json')

print('Run time:', (time.time() - start_time) / 60, 'min')

Run time: 1.6807258168856303 min
